## Proyecto Análisis de Datos

En la clase de hoy vamos a ver un Proyecto completo de Análisis de Datos completo realizando todos los pasos necesarios:

* Definición del problema
* Búsqueda de datos
* Análisis exploratorio
* Limpieza
* Visualizaciones
* Presentación de resultados


### Definición del Problema

En este caso queremos conocer información sobre las cuentas de instagram que mayor cantidad de seguidores tienen

### Búsqueda de datos

Utilizaramos un dataset extraído de Kaggle que cuenta con los datos de las 200 cuentas de instagram con mayores seguidores en el año 2023. extraído de https://www.kaggle.com/datasets/faisaljanjua0555/top-200-most-followed-instagram-accounts-2023

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv("data.csv")

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
# Vamos a reenombras las columnas al español

data.columns

In [ ]:
data.columns = ["rango", "nombre", "descripcion", "categoria", "publicaciones", "seguidores", "me_gusta_promedio", "tasa_participacion"]

In [ ]:
data.columns

In [ ]:
data.dtypes

#### Columna *descripcion*

In [ ]:
# Vamos a explorar la distribución de datos de descripcion

data["descripcion"].value_counts()

In [ ]:
# Vamos a explorar la distribución de datos de categoria

data["categoria"].value_counts()

#### Columnas *publicaciones*, *seguidores*, *tasa_participacion* y *me_gusta_promedio*

Poder ver como un problema es que los datos numéricos los está tomando como *object*, vemos que el problema es que está en el número y luego "k"/ "m" o "%" y no lo puede leer como nùmero. Lo que vamos a realizar es reemplazar esas letras y símbolos por números utilizando *apply* y *lambda* y luego cambiar el tipo de dato en cada una de las columnas utilizando el método [astype](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html)

In [ ]:
# luego reemplazamos K por 00 (el tercer cero no se coloca porque quitamos los puntos decimales)

data["publicaciones"] = data["publicaciones"].apply(lambda x: x.replace("K", ""))

In [ ]:
# Luego convertimos el valor en numérico

data["publicaciones"]=data["publicaciones"].astype(float)

In [ ]:
# Multiplicamos los valores por 1000

data["publicaciones"] = data["publicaciones"]*1000

In [ ]:
data["publicaciones"]

In [ ]:
data["seguidores"] = data["seguidores"].apply(lambda x: x.replace("M", ""))
data["seguidores"]=data["seguidores"].astype(float)
data["seguidores"] = data["seguidores"]*1000000

In [ ]:
data["seguidores"]

In [ ]:
data["me_gusta_promedio"] = data["me_gusta_promedio"].apply(lambda x: x.replace("K", ""))
data["me_gusta_promedio"]=data["me_gusta_promedio"].astype(float)
data["me_gusta_promedio"] = data["me_gusta_promedio"]*1000

In [ ]:
data["me_gusta_promedio"]

In [ ]:
data["tasa_participacion"] = data["tasa_participacion"].apply(lambda x: x.replace("%", ""))
data["tasa_participacion"]=data["tasa_participacion"].astype(float)

In [ ]:
data["tasa_participacion"]

In [ ]:
data.dtypes

In [ ]:
# Revisamos los datos nulos

data.isnull().sum()

In [ ]:
# Primero revisamos la columan descripcion

mask_nulo_descripcion = data["descripcion"].isnull()
mask_nulo_descripcion

In [ ]:
data[mask_nulo_descripcion]

Podemos considerar que michelleobama es "female" ylos demàs ("jungkook.97", "uarmyhope", "agustd" y "tyga") son "male" (por el conocimiento sobre el dataset por parte del Analista de Datos. Vamos a **imputar** los valores correspondientes)

In [ ]:
data.loc[104, "descripcion"]

In [ ]:
data.loc[104, "descripcion"] = "female"

In [ ]:
lista_filas = [118,147,158,177]

for i in lista_filas:
    data.loc[i, "descripcion"] = "male"

In [ ]:
# revisamos la operación

data[mask_nulo_descripcion]

In [ ]:
data.isnull().sum()

In [ ]:
# Ahora revisamos categoria

data[data["categoria"].isnull()]


Como no sabemos exactamente cual es la categría, podemos dejarlos nulos o imputarlos por datos aleatorios, en este caso utilizaremos un método de pandas [fillna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) que reemplaza los valores nulos, en este caso vamos a indicar que por la fila superior.

In [ ]:
data["categoria"].fillna(method="ffill", inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data.describe()

#### Método *to_csv*

Para poder volver a utilizar el Dataset con las modificaciones realizadas vamos a utilizar el método [to_csv](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)

In [ ]:
# Podemos utilizar el método

data.to_csv("instagram_limpio.csv", index=False)


#### Visualizaciones

In [ ]:
data["descripcion"].value_counts()

In [ ]:
# Vamos a hacer un gráfico countplot con los datos de descripcion

plt.pie(data["descripcion"].value_counts(), labels=["male", "female", "brand", "community"], autopct = '%0.0f%%')

In [ ]:
# Vamos a hacer un gráfico countplot con los datos de categoria

plt.figure(figsize=(10, 5))
sns.countplot(data["categoria"], order=data["categoria"].value_counts().index)
plt.xticks(rotation=45)

In [ ]:
# Distribución de variable seguidores

sns.distplot(data["seguidores"])

In [ ]:
# Vamos a gráficar el top 10 en seguidores, para eso tenemos que hacer una agrupación:

sns.barplot(data=data[:10], x="seguidores", y="nombre", palette="hsv")
plt.title("Top cuentas de Instagram por seguidores")
plt.xlabel("cantidad de seguidores")
plt.ylabel("cuenta de instragram")

In [ ]:
# Para graficar el top 10 de me gusta, tenemos que primero reordenar el dataset
top_me_gusta=data.sort_values(by="me_gusta_promedio", ascending=False)[:10]
top_me_gusta


In [ ]:
sns.barplot(data=top_me_gusta[:10], x="me_gusta_promedio", y="nombre", palette="hsv")
plt.title("Top cuentas de Instagram por me gusta")
plt.xlabel("cantidad de me gusta promedio")
plt.ylabel("cuenta de instragram")

In [ ]:
# Vamos a graficar la relacion entre la cantidad de publicaciones y de seguidores con un scatterplot

sns.scatterplot(data=data, x="publicaciones", y="seguidores")


In [ ]:
# Vamos a graficar la relación entre las categorias y la cantidad de seguidores promedio para eso hay que hacer una agrupación

data_categoria_seguidores = data.groupby("categoria")["seguidores"].mean().sort_values(ascending=False)

In [ ]:
sns.barplot(x=data_categoria_seguidores, y=data_categoria_seguidores.index)

In [ ]:
ax = data_categoria_seguidores.plot.barh(stacked=True)
plt.savefig("Seguidores x Categoria.jpg")

#fig = ax.get_figure()
#fig.savefig("output.png")
#plt.savefig('output.png')
#sns_plot.savefig("output.png")
